In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras import Model

In [2]:
df = pd.read_csv('./data/clean_data_with_date_weather.csv', index_col=0)
df

,index,county,state,season,title,latitude,longitude,date,number,geohash,...,dew_point,humidity,cloud_cover,precip_type,pressure,wind_bearing,wind_speed,Class A,Class B,Class C
0,2,Washington County,Rhode Island,Fall,Report 6496: Bicycling student has night encou...,41.45000,-71.50000,1974-09-20,6496.0,drm5ucxrc0,...,66.227150,0.907111,0.268375,rain,1019.476257,263.869568,11.258579,1,0,0
1,5,Washita County,Oklahoma,Fall,Report 9765: Motorist and children have daylig...,35.30110,-99.17020,1973-09-28,9765.0,9y32z667yc,...,47.139355,0.646660,0.113750,No Precipitation,953.307434,164.583038,14.922651,1,0,0
2,6,Washington County,Ohio,Summer,Report 4983: Sighting and vocalizations at a h...,39.38745,-81.67339,1971-08-01,4983.0,dpjbj6r280,...,60.795803,0.745234,0.584542,No Precipitation,993.191711,17.920425,4.966946,1,0,0
3,7,Westchester County,New York,Fall,Report 31940: Fishermen and Law Enforcement Of...,41.29873,-73.73717,2010-09-01,31940.0,dr7dfn4nx3,...,63.793255,0.578789,0.126625,No Precipitation,1001.331787,270.616058,6.397976,1,0,0
4,8,Washoe County,Nevada,Fall,Report 5692: Father and son hear vocalizations...,39.59200,-119.97500,1970-09-01,5692.0,9r49jzsb5g,...,29.029099,0.402105,0.000000,No Precipitation,807.823975,219.053116,9.812248,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,5010,Harlan County,Kentucky,Fall,Report 25257: ATV riders find possible tracks ...,36.89771,-83.16293,2008-10-15,25257.0,dnscgt48er,...,54.413593,0.740113,0.151125,No Precipitation,911.663757,201.991241,5.410581,0,1,0
4035,5011,Harlan County,Kentucky,Winter,Report 38980: Teen on four-wheeler has sightin...,36.83193,-83.42429,2013-01-10,38980.0,dns9t9h391,...,47.541798,0.930158,0.878042,rain,980.778503,89.679916,6.219504,1,0,0
4036,5012,Anderson County,Kentucky,Fall,Report 1045: Father and son have a daylight en...,37.95708,-84.97527,2005-01-09,1045.0,dnexfmfuh8,...,33.796551,0.793257,0.548708,No Precipitation,995.131042,178.840332,7.880466,1,0,0
4037,5013,Anderson County,Kentucky,Fall,Report 33187: Dual sighting in late afternoon ...,37.97699,-84.90234,2011-11-15,33187.0,dng8h18000,...,56.680698,0.936365,0.963083,rain,984.935791,201.164108,13.016956,1,0,0


In [3]:
def one_hot_encode(data, col_name):
    data = pd.concat([data, pd.get_dummies(data[col_name], dtype=int)], axis=1)
    data.drop(col_name, axis=1, inplace=True)
    return data

In [4]:
season_dict = {
    '01': 'Winter',
    '02': 'Winter',
    '03': 'Spring',
    '04': 'Spring',
    '05': 'Spring',
    '06': 'Summer',
    '07': 'Summer',
    '08': 'Summer',
    '09': 'Fall',
    '10': 'Fall',
    '11': 'Fall',
    '12': 'Winter',
}

season_col = df.columns.get_loc('season')
for row_idx in df[df['season'] == 'Unknown'].index:
    month = df.iloc[row_idx, :]['date'].split('-')[1]
    df.iloc[row_idx, season_col] = season_dict[month]

In [5]:
# Convert to unix timestamp
df['date'] = df['date'].astype(np.datetime64).astype('int64')/1e9

In [56]:
x_df = df.drop(['index', 'county', 'state', 'title', 'geohash'], axis=1)
x_df = one_hot_encode(x_df, 'season')
x_df = one_hot_encode(x_df, 'precip_type')
x_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4039 entries, 0 to 4038
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   latitude          4039 non-null   float64
 1   longitude         4039 non-null   float64
 2   date              4039 non-null   float64
 3   number            4039 non-null   float64
 4   temperature_high  4039 non-null   float64
 5   temperature_mid   4039 non-null   float64
 6   temperature_low   4039 non-null   float64
 7   dew_point         4039 non-null   float64
 8   humidity          4039 non-null   float64
 9   cloud_cover       4039 non-null   float64
 10  pressure          4039 non-null   float64
 11  wind_bearing      4039 non-null   float64
 12  wind_speed        4039 non-null   float64
 13  Class A           4039 non-null   int64  
 14  Class B           4039 non-null   int64  
 15  Class C           4039 non-null   int64  
 16  Fall              4039 non-null   int32  


In [57]:
x_train, x_test = train_test_split(x_df, test_size = 0.2)

latitude_train = np.array(x_train['latitude'])
longitude_train = np.array(x_train['longitude'])

latitude_test = np.array(x_test['latitude'])
longitude_test = np.array(x_test['longitude'])

x_train = x_train.drop(['latitude', 'longitude'], axis=1)
x_test = x_test.drop(['latitude', 'longitude'], axis=1)

# Normalize inputs
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [58]:
def create_model(input_len, optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), loss_type='mse'):
    # https://manuel-gilm.medium.com/multi-target-regression-with-tensorflow-a7d974cf5d89
    input_layer = Input(shape=input_len)
    dense_layer_1 = Dense(units=128, activation="relu")(input_layer) 
    dense_layer_2 = Dense(units=128, activation="relu")(dense_layer_1)
    dense_layer_3 = Dense(units=64, activation="relu")(dense_layer_2)
    
    latitude_output = Dense(units=1, activation="linear", name="latitude_output")(dense_layer_2)
    longitude_output = Dense(units=1, activation="linear", name="longitude_output")(dense_layer_3)
    
    model = Model(inputs=input_layer, outputs=[latitude_output, longitude_output])
        
    model.compile(optimizer = optimizer,
                  loss = {'latitude_output': loss_type, 'longitude_output': loss_type},
                  metrics = {
                      'latitude_output': tf.keras.metrics.RootMeanSquaredError(),
                      'longitude_output': tf.keras.metrics.RootMeanSquaredError(),
                  }
                 )
    
    return model

In [59]:
x = np.array(x_train)
y = np.array([latitude_train, longitude_train])
x_test = np.array(x_test)
y_test = np.array([latitude_test, longitude_test])

In [55]:
model = create_model(x_train.shape[1], tf.keras.optimizers.Adam(learning_rate=0.001))
history = model.fit(x, y.T, epochs=100, batch_size=32,
                    validation_data = (x_test, y_test.T), verbose=1)

Epoch 1/100
101/101 [==============================] - 1s 6ms/step - loss: 10101.1689 - latitude_output_loss: 5060.1167 - longitude_output_loss: 5041.0513 - latitude_output_root_mean_squared_error: 71.1345 - longitude_output_root_mean_squared_error: 71.0004 - val_loss: 9651.7715 - val_latitude_output_loss: 4826.3242 - val_longitude_output_loss: 4825.4453 - val_latitude_output_root_mean_squared_error: 69.4717 - val_longitude_output_root_mean_squared_error: 69.4654
Epoch 2/100
101/101 [==============================] - 1s 5ms/step - loss: 9653.2207 - latitude_output_loss: 4826.5366 - longitude_output_loss: 4826.6826 - latitude_output_root_mean_squared_error: 69.4733 - longitude_output_root_mean_squared_error: 69.4743 - val_loss: 9635.0459 - val_latitude_output_loss: 4817.5762 - val_longitude_output_loss: 4817.4697 - val_latitude_output_root_mean_squared_error: 69.4088 - val_longitude_output_root_mean_squared_error: 69.4080
Epoch 3/100
101/101 [==============================] - 0s 5ms/ste

In [48]:
model = create_model(x_train.shape[1], tf.keras.optimizers.Adam(learning_rate=0.001), loss_type='mae')
history = model.fit(x, y.T, epochs=100, batch_size=32,
                    validation_data = (x_test, y_test.T), verbose=1)

Epoch 1/100
101/101 [==============================] - 1s 7ms/step - loss: 136.6396 - latitude_output_loss: 68.3198 - longitude_output_loss: 68.3198 - latitude_output_mean_absolute_error: 68.3198 - longitude_output_mean_absolute_error: 68.3198 - val_loss: 137.4212 - val_latitude_output_loss: 68.7106 - val_longitude_output_loss: 68.7106 - val_latitude_output_mean_absolute_error: 68.7106 - val_longitude_output_mean_absolute_error: 68.7106
Epoch 2/100
101/101 [==============================] - 1s 5ms/step - loss: 136.6395 - latitude_output_loss: 68.3198 - longitude_output_loss: 68.3198 - latitude_output_mean_absolute_error: 68.3198 - longitude_output_mean_absolute_error: 68.3198 - val_loss: 137.4212 - val_latitude_output_loss: 68.7106 - val_longitude_output_loss: 68.7106 - val_latitude_output_mean_absolute_error: 68.7106 - val_longitude_output_mean_absolute_error: 68.7106
Epoch 3/100
101/101 [==============================] - 0s 4ms/step - loss: 136.6395 - latitude_output_loss: 68.3198 - 

In [49]:
model = create_model(x_train.shape[1], loss_type='mae')
history = model.fit(x, y.T, epochs=100, batch_size=32,
                    validation_data = (x_test, y_test.T), verbose=1)

Epoch 1/100
101/101 [==============================] - 1s 5ms/step - loss: 136.6396 - latitude_output_loss: 68.3198 - longitude_output_loss: 68.3198 - latitude_output_mean_absolute_error: 68.3198 - longitude_output_mean_absolute_error: 68.3198 - val_loss: 137.4212 - val_latitude_output_loss: 68.7106 - val_longitude_output_loss: 68.7106 - val_latitude_output_mean_absolute_error: 68.7106 - val_longitude_output_mean_absolute_error: 68.7106
Epoch 2/100
101/101 [==============================] - 0s 5ms/step - loss: 136.6395 - latitude_output_loss: 68.3198 - longitude_output_loss: 68.3198 - latitude_output_mean_absolute_error: 68.3198 - longitude_output_mean_absolute_error: 68.3198 - val_loss: 137.4212 - val_latitude_output_loss: 68.7106 - val_longitude_output_loss: 68.7106 - val_latitude_output_mean_absolute_error: 68.7106 - val_longitude_output_mean_absolute_error: 68.7106
Epoch 3/100
101/101 [==============================] - 0s 4ms/step - loss: 136.6395 - latitude_output_loss: 68.3198 - 

In [41]:
model = create_model(x_train.shape[1], tf.keras.optimizers.SGD(learning_rate=0.001))
history = model.fit(x, y.T, epochs=100, batch_size=32,
                    validation_data = (x_test, y_test.T), verbose=1)

Epoch 1/100
101/101 [==============================] - 1s 7ms/step - loss: 9740.1914 - latitude_output_loss: 4875.5161 - longitude_output_loss: 4864.6748 - latitude_output_root_mean_squared_error: 69.8249 - longitude_output_root_mean_squared_error: 69.7472 - val_loss: 9728.1982 - val_latitude_output_loss: 4863.7485 - val_longitude_output_loss: 4864.4487 - val_latitude_output_root_mean_squared_error: 69.7406 - val_longitude_output_root_mean_squared_error: 69.7456
Epoch 2/100
101/101 [==============================] - 0s 5ms/step - loss: 9619.9775 - latitude_output_loss: 4807.9990 - longitude_output_loss: 4811.9766 - latitude_output_root_mean_squared_error: 69.3397 - longitude_output_root_mean_squared_error: 69.3684 - val_loss: 9726.1035 - val_latitude_output_loss: 4862.8467 - val_longitude_output_loss: 4863.2573 - val_latitude_output_root_mean_squared_error: 69.7341 - val_longitude_output_root_mean_squared_error: 69.7371
Epoch 3/100
101/101 [==============================] - 0s 4ms/step

In [ ]:
# https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/